# Calculate the volume for the Delta using the h5 hydro information

In [ ]:
import pandas as pd

import pydsm

from pydsm import hydroh5

In [ ]:
hydro_tidefilename = 'd:/delta/dsm2_studies_master/studies/historical/output/hist_v2022_01.h5'

In [ ]:
hydro = hydroh5.HydroH5(hydro_tidefilename)

## Channel ids 

In [ ]:
cids = list(hydro.get_channels().values.flatten())

## Timewindow

In [ ]:
timewindow = '01JAN2014 - 01JAN2015'

## Channel Areas

In [ ]:
chan_areas = hydro.get_channel_avg_area(cids,timewindow)
chan_areas

## Channel Lengths

In [ ]:
channels = hydro.get_input_table('/hydro/input/channel')

chan_lengths = channels.set_index('chan_no')['length']
chan_lengths

## Channel Volumes = Channel Lengths * Channel Areas

In [ ]:
chan_vols = chan_lengths.values*chan_areas

# Visualization

In [ ]:
import hvplot.pandas

In [ ]:
# 1 acre-feet = 43559.9 cubic feet
# 1 maf (million acre-feet) = 43559.9*1e6
maf_factor = 43559.9*1e6

In [ ]:
chan_vols_maf = chan_vols/maf_factor

chan_vols_maf.sum(axis=1).hvplot(title='Sum of Channel Volumes', ylabel='Channel Volume (Million Acre-Feet)')

# Volume of Reservoirs = Area of Reservoirs * Height of water in Reservoir

In [ ]:
hydro.get_reservoirs()

list(hydro.get_reservoirs().values.flatten())

reservoirs_height = hydro.get_reservoir_height(
    list(hydro.get_reservoirs().values.flatten()), timewindow)
reservoirs_height

reservoirs = hydro.get_input_table('/hydro/input/reservoir')
reservoirs

# reservoir areas are in millions of square feet
reservoirs_volume = (reservoirs.area.values*1e6)*reservoirs_height

In [ ]:
reservoirs_volume_maf = reservoirs_volume/maf_factor

In [ ]:
reservoirs_volume_maf.sum(axis=1).hvplot(title='Sum of Reservoir Volumes', ylabel = 'Volume (Million Acre-Feet)')

In [ ]:
dsm2_full_grid_vol = chan_vols_maf.sum(axis=1)+reservoirs_volume_maf.sum(axis=1)
dsm2_full_grid_vol.hvplot(title='DSM2 Volume (Full Extent)',ylabel='Volume (million acre-feet)')

# Volume of a subset of channels
In this case, the channels in the Suisun Marsh

In [ ]:
chans_suisun_marsh = pd.read_csv('suisun_marsh_channel_numbers.csv')

chans_suisun_marsh.channel_nu.values

suisun_marsh_cols = chan_vols_maf.columns[
    chan_vols_maf.columns.astype(int).isin(chans_suisun_marsh.channel_nu.values)]
not_suisun_marsh_cols = chan_vols_maf.columns[~
    chan_vols_maf.columns.astype(int).isin(chans_suisun_marsh.channel_nu.values)]


chan_vols_maf[suisun_marsh_cols].sum(axis=1).hvplot(title='DSM2 Volume (Suisun Marsh)',ylabel='Volume (million acre-feet)')

In [ ]:
(chan_vols_maf[not_suisun_marsh_cols].sum(
    axis=1)+reservoirs_volume_maf.sum(
    axis=1)).hvplot(title='DSM2 Volume (Delta only)',ylabel='Volume (million acre-feet)')